In [2]:
import os
import requests
import json
import pandas as pd

# Define the endpoint URL for bike stations
url = "https://api.foursquare.com/v3/places/search"
params = {
    'query': 'bike station',
    'location': 'Toronto',
    'radius': 1000
}

headers = {
        "Accept": "application/json",
        "Authorization": "fsq3sBRRp8FffG0vdAxZKDQyxmSiK9QoQUhBBYoVNItiqjw=" 
    }

# Send GET request to Foursquare for bike stations
response = requests.get(url, params=params, headers=headers)

# Check if the status code is 200 (OK)
if response.status_code == 200:
    print('The status code is 200')
    foursquare_bike = json.loads(response.text)

    # Parse through the response to get the bike station details
    bike_stations = []
    for result in foursquare_bike['results']:
        station = {
            'name': result['name'],
            'location': result['location']['formatted_address']
        }
        bike_stations.append(station)

    # Create DataFrame for bike stations
    bike_stations_df = pd.DataFrame(bike_stations)
    print(bike_stations_df)
else:
    print('Request failed with status code', response.status_code)


# Define the endpoint URL for parks
endpoint = "https://api.foursquare.com/v3/places/search"
params = {
    'term': 'park',
    'location': 'Toronto'
}

# Send GET request to Foursquare for parks
response = requests.get(endpoint, params=params, headers=headers)

# Check if the status code is 200 (OK)
if response.status_code == 200:
    print('The status code is 200')
    foursquare_park = json.loads(response.text)

    # Parse through the response to get the park details
    parks = []
    for result in foursquare_park['results']:
        park = {
            'name': result['name'],
            'location': result['location']['formatted_address']
        }
        parks.append(park)

    # Create DataFrame for parks
    parks_df = pd.DataFrame(parks)
    print(parks_df)
else:
    print('Request failed with status code', response.status_code)


The status code is 200
                    name                                           location
0  Cooksville GO Station  3210 Hurontario St (at Hillcrest Ave), Mississ...
The status code is 200
                                 name  \
0  The Apple Market-Tiveron Farms Ltd   
1                         Whole Foods   
2            French Corner Patisserie   
3                      Crate & Barrel   
4                      Door FiftyFive   
5    Port Credit Business Association   
6                         Dairy Cream   
7           Lezzet Shawarma Falafel H   
8                               Saucy   
9                            Chapters   

                                            location  
0  2281 Camilla Rd (at Queensway E), Mississauga ...  
1  155 Sq One Dr (at Hammerson Dr), Mississauga O...  
2  1224 Dundas St W (Just west of Erindale Statio...  
3  95 Sq One Dr (at Square One Mall), Mississauga...  
4          55 Lakeshore Rd E, Mississauga ON L5G 1C9  
5         105 Lakesh

In [3]:
import os
import requests
import json
import pandas as pd

business_id = 'srJ-hGeN_7H5oGT4wYz7uw'
api_key = "w7GSO5abCgHWgnZ-Vd21P00zNIY_nx-1ygAPL-5XngiWsGeBAPuTaMVPSB4ulo3veq0cjeEiPtM9IzErrIWwwrI3Wt5ELsCrx5qT4xDp6UgHBe83PU7LebN4MRx-ZHYx"
headers = {'Authorization': 'Bearer %s' % api_key}

# Define the endpoint URL for bike stations
endpoint = 'https://api.yelp.com/v3/businesses/search'
params = {'term': 'Bike stations', 'location': 'Toronto', 'radius': 1000}

# Send GET request to Yelp API for bike stations
response = requests.get(endpoint, params=params, headers=headers)

# Check if the status code is 200 (OK)
if response.status_code == 200:
    print('The status code is 200')
    bike_yelp_details = json.loads(response.text)

    # Use json_normalize() to convert JSON to DataFrame
    df_bike_stations = pd.json_normalize(bike_yelp_details['businesses'])
    print(df_bike_stations)
else:
    print('Request failed with status code', response.status_code)


# Define the endpoint URL for restaurants
endpoint = 'https://api.yelp.com/v3/businesses/search'
params = {'term': 'Restaurants', 'location': 'Toronto'}

# Send GET request to Yelp API for restaurants
response = requests.get(endpoint, params=params, headers=headers)

# Check if the status code is 200 (OK)
if response.status_code == 200:
    print('The status code is 200')
    restuarnt_details = json.loads(response.text)

    # Use json_normalize() to convert JSON to DataFrame
    df_restaurants = pd.json_normalize(restuarnt_details['businesses'])
    restaurant_data = df_restaurants[['name', 'review_count', 'location.address1', 'rating']]
    print(restaurant_data)
else:
    print('Request failed with status code', response.status_code)

# Sort the restaurant data by rating in descending order
sorted_restaurant_data = restaurant_data.sort_values(by='rating', ascending=False)
print(sorted_restaurant_data.head(10))


The status code is 200
Empty DataFrame
Columns: []
Index: []
The status code is 200
                            name  review_count         location.address1  \
0      Pai Northern Thai Kitchen          3397          18 Duncan Street   
1                           Mira           263  420A Wellington Street W   
2               Richmond Station           887       1 Richmond Street W   
3                           Rasa           321         196 Robert Street   
4                        Katsuya           488       66 Gerrard Street E   
5                        Aloette           287        163 Spadina Avenue   
6                      Gusto 101          1122       101 Portland Street   
7              Trattoria Nervosa           390       75 Yorkville Avenue   
8                 Yeah Yeah Thai            12        382 Spadina Avenue   
9                     Fieramosca           110  36A Prince Arthur Avenue   
10                         Irene             1         25 Dockside Drive   
11  